### 프로젝트 수행에 필요한 라이브러리 설치

In [ ]:
# 라이브러리 설치
!pip install numpy==1.23.5
!pip install torch==2.7.0
!pip install scikit-learn==1.6.1
!pip install tqdm==4.67.1
!pip install matplotlib==3.9.4
!pip install sentencepiece==0.1.96
!pip install transformers==4.52.3
!pip install mxnet -f https://dist.mxnet.io/python/cpu
!pip install gluonnlp==0.9.1
!pip install konlpy==0.6.0
!pip install nltk==3.9.1
!pip install hgtk==0.2.1
!pip install jamo==0.4.1
!pip install hangul-utils==0.2
!pip install xgboost==2.1.4
!pip install googletrans==4.0.0rc1
!pip install git+https://github.com/Seokhyeon-Park/hanspell.git
!pip install confusion-matrix==0.1
!pip install tensorflow==2.12.0
#!pip install mitmproxy
!pip install selenium==4.33.0
!pip install webdriver-manager==4.0.2
!pip install pandas==2.2.3
!pip install blinker==1.6.2
!pip install selenium-wire==5.1.0
#!pip install komoran
!pip install WordCloud==1.9.4
!pip install seaborn==0.13.2

In [ ]:
nltk.download('words')
nltk.download('punkt')  
nltk.download('punkt_tab') # 토큰화 도구(최초 1회만 다운로드 필요)
nltk.download('averaged_perceptron_tagger')

### passport key 파싱을 위한 필요 라이브러리 호출

In [2]:
import re
import os
import unicodedata
import hanspell

from selenium import webdriver
from seleniumwire import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from seleniumwire.utils import decode
import urllib.parse as urlparse
from urllib.parse import parse_qs
import time
from selenium.webdriver.common.by import By

### 네이버 맞춤법 검사기에서 passport key 파싱 후 파일에 자동 적용

In [ ]:
options=Options()
service=Service(ChromeDriverManager().install())
driver=webdriver.Chrome(service=service,options=options)
driver.get("https://search.naver.com/search.naver?query=맞춤법+검사기")
time.sleep(5)
input_box = driver.find_element(By.CSS_SELECTOR, "textarea.txt_gray")
input_box.click()
time.sleep(3)
input_box = driver.find_element(By.CSS_SELECTOR, "textarea.stand_txt")
input_box.send_keys("예시 문장입니다.")
submit_btn = driver.find_element(By.CSS_SELECTOR, "button.btn_check")

submit_btn.click()
time.sleep(2)

for request in driver.requests:
    if "SpellerProxy" in request.url :
        parsed = urlparse.urlparse(request.url)
        passport_key = parse_qs(parsed.query)['passportKey'][0]
        underber = parse_qs(parsed.query)['_'][0]
        break

driver.quit()

hanspell_path = os.path.dirname(hanspell.__file__)
spell_checker_path = os.path.join(hanspell_path, 'spell_checker.py')

print("spell_checker.py 파일 위치:", spell_checker_path)

with open(spell_checker_path, 'r', encoding='utf-8') as f:
    src = f.read()

pattern = r"'passportKey': '.*'"
print(passport_key)
src = re.sub(pattern, f"'passportKey': '{passport_key}'", src)
with open(spell_checker_path, 'w', encoding='utf-8') as f:
    f.write(src)

print("spell_checker.py 자동 수정 완료!")

### 프로그램 종료 후 main.ipynb 에서 재실행